In [29]:
import pandas as pd
import pyodbc
connection_string = ("DRIVER={SQL Server};"
"SERVER=USDF11V0954;"
"DATABASE=Actuarial_AH;"
"Trust_Connection=yes;")

conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [28]:
med = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
    WHERE ART_DRUGGRPS_L2 IN ('GLP-1 Agonists', 'Weight Loss Agents')
    AND SERVICE_DATE >= '2021-01-01'
    AND SERVICE_DATE < '2025-01-01'
),
WEIGHT_DX AS (
    SELECT DISTINCT MEMBER_ID
    FROM Actuarial_AH.DBO.SN_Medical
    WHERE ICD10_SECTION LIKE '%Overweight%'
)

SELECT M.MEDICAL_CLAIM_ID, 
       M.MEMBER_ID, 
       M.PAID_DATE, 
       YEAR(M.PAID_DATE) AS PAID_YEAR,
       M.SERVICE_DATE,
       YEAR(M.SERVICE_DATE) AS SERVICE_YEAR,
       M.MEDICAL_PAID_AMOUNT, 
       M.IS_TELEMEDICINE, 
       M.IS_ER_AVOIDABLE, 
       M.DIAGNOSIS_DESC_ICD10_1 AS PRIMARY_DX,
       M.PROCEDURE_DESC, 
       M.ARTTOS_V2_L1, 
       M.ARTTOS_V2_L3, 
       M.DX_IS_CHRONIC,
       M.ICD10_CHAPTER, 
       M.ICD10_CATEGORY, 
       M.ICD10_SECTION, 
       M.MEG_EPISODE_DESCRIPTION, 
       M.OP_SURG_INC,
       M.IS_PCP_VISIT, 
       M.INCLUDED_SPECIALIST, 
       M.ER_VISIT_FLAG, 
       M.IS_URGENT_CARE_VISIT, 
       M.IS_PREVENTIVE_VISIT, 
       M.IP_ADMIT_INC
FROM [Actuarial_AH].[dbo].[SN_Medical] M
JOIN Actuarial_AH.dbo.SN_Member MB ON MB.MEMBER_ID = M.MEMBER_ID
WHERE M.PAID_DATE BETWEEN '2021-01-01' AND '2024-12-31'
AND MB.MONTH_KEY = '2024-12-01' 
AND MB.MEDICAL_ENROLLMENT_STATUS = 3
AND MB.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND MB.MEMBER_ID NOT IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
)
AND MB.MEMBER_ID IN (
    SELECT MEMBER_ID
    FROM WEIGHT_DX
)
"""
med = pd.read_sql(med, conn)

NameError: name 'conn' is not defined

In [4]:
rx = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
WHERE ART_DRUGGRPS_L2 IN ('GLP-1 Agonists', 'Weight Loss Agents')
    AND SERVICE_DATE BETWEEN '2021-01-01' AND '2024-12-31'
),
WEIGHT_DX AS (
    SELECT DISTINCT MEMBER_ID
    FROM Actuarial_AH.DBO.SN_Medical
    WHERE ICD10_SECTION LIKE '%Overweight%'
)
SELECT R.RX_CLAIM_ID, 
       R.MEMBER_ID, 
       R.PAID_DATE, 
       R.SERVICE_DATE, 
       R.DRUG_NAME_GENERIC,
	   R.DRUG_NAME_PREFERRED,
	   R.RX_PAID_AMOUNT,
	   R.RX_SCRIPT_COUNT,
	   R.ART_DRUGGRPS_L1,
	   R.ART_DRUGGRPS_L2,
	   YEAR(R.PAID_DATE) AS PAID_YEAR,
YEAR(R.SERVICE_DATE) AS SERVICE_YEAR
FROM [Actuarial_AH].[dbo].[SN_Rx] R
JOIN 
Actuarial_AH.dbo.SN_Member M ON R.MEMBER_ID = M.MEMBER_ID
JOIN Actuarial_AH.dbo.SN_Medical MD ON MD.MEMBER_ID = M.MEMBER_ID
WHERE R.PAID_DATE BETWEEN '2021-01-01' AND '2024-12-31'
AND M.MONTH_KEY = '2024-12-01' 
AND M.MEDICAL_ENROLLMENT_STATUS = 3
AND M.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND M.MEMBER_ID NOT IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
	)
AND MD.ICD10_SECTION LIKE '%Overweight%'
"""
rx = pd.read_sql(rx, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_26976\2795320354.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rx = pd.read_sql(rx, conn)


In [5]:
member = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
WHERE ART_DRUGGRPS_L2 IN ('GLP-1 Agonists', 'Weight Loss Agents')
    AND SERVICE_DATE >= '2022-01-01'
    AND SERVICE_DATE < '2025-01-01'
),
WEIGHT_DX AS (
    SELECT DISTINCT MEMBER_ID
    FROM Actuarial_AH.DBO.SN_Medical
    WHERE ICD10_SECTION LIKE '%Overweight%'
)
SELECT M.MEMBER_ID,
       M.MEMBER_STATUS,
       M.HOME_STATE,
	   CASE 
    WHEN M.MEMBER_RELATIONSHIP = 0 THEN 'Subscriber'
    WHEN M.MEMBER_RELATIONSHIP = 1 THEN 'Spouse'
    WHEN M.MEMBER_RELATIONSHIP = 2 THEN 'Dependent'
    WHEN M.MEMBER_RELATIONSHIP = 3 THEN 'Domestic Partner'
    WHEN M.MEMBER_RELATIONSHIP = 4 THEN 'Unknown'
    ELSE 'Unknown' -- Optional: Handle unexpected values
END AS MEMBER_RELATIONSHIP,
CASE 
        WHEN M.GENDER = 0 THEN 'Female'
        WHEN M.GENDER = 1 THEN 'Male'
        WHEN M.GENDER = 2 THEN 'Unknown'
        ELSE 'Unknown' 
    END AS GENDER,
    DATEDIFF(YEAR, CONVERT(DATE, M.BIRTH_YEAR_MO + '-01'), GETDATE()) - 
    CASE 
        WHEN MONTH(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) > MONTH(GETDATE()) 
             OR (MONTH(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) = MONTH(GETDATE()) 
                 AND DAY(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) > DAY(GETDATE())) 
        THEN 1 
        ELSE 0 
    END AS Age,
	CASE 
    WHEN M.BIRTH_YEAR >= 1946 AND M.BIRTH_YEAR < 1965 THEN 'Baby Boomers'
	WHEN M.BIRTH_YEAR >= 1965 AND M.BIRTH_YEAR < 1981 THEN 'Generation X'
	WHEN M.BIRTH_YEAR >= 1981 AND M.BIRTH_YEAR < 1997 THEN 'Millenials'
	WHEN M.BIRTH_YEAR >= 1997 AND M.BIRTH_YEAR < 2012 THEN 'Generation Z'
    ELSE 'Unknown' -- Optional: Handle unexpected values
END AS GENERATIONS
FROM [Actuarial_AH].[dbo].[SN_MEMBER] M
JOIN Actuarial_AH.dbo.SN_Medical MD ON M.MEMBER_ID = MD.MEMBER_ID
WHERE M.MEMBER_ID NOT IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
	)
AND M.MONTH_KEY = '2024-12-01' 
AND M.MEDICAL_ENROLLMENT_STATUS = 3
AND M.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND MD.ICD10_SECTION LIKE '%Overweight%'

"""
member = pd.read_sql(member, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_26976\4259235446.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  member = pd.read_sql(member, conn)


In [6]:
member

,MEMBER_ID,MEMBER_STATUS,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS
0,mma-c1d477e010df4d1ab387a9bc4d01d6fd,0.0,CT,Subscriber,Male,34,Millenials
1,mma-036ffc20ff794595a44950a4729f66d3,0.0,LA,Subscriber,Female,62,Baby Boomers
2,mma-036ffc20ff794595a44950a4729f66d3,0.0,LA,Subscriber,Female,62,Baby Boomers
3,mma-4c45de73f32445699b80d8d493793dff,0.0,MA,Subscriber,Male,57,Generation X
4,mma-4c45de73f32445699b80d8d493793dff,0.0,MA,Subscriber,Male,57,Generation X
...,...,...,...,...,...,...,...
154559,mma-565511f8a56541379d9fd2f16e419df3,0.0,IL,Subscriber,Female,64,Baby Boomers
154560,mma-565511f8a56541379d9fd2f16e419df3,0.0,IL,Subscriber,Female,64,Baby Boomers
154561,mma-cb4e69f3bcba4d6fa609f5e4be267b0a,0.0,NC,Spouse,Female,42,Millenials
154562,mma-e05f6060ee6c448ebee94236610c8ebb,0.0,KY,Subscriber,Male,45,Generation X


In [7]:
member.to_csv('no-glp-1-member-claims.csv', index=False)
med.to_csv('med-no-glp-claims.csv', index=False)
rx.to_csv('rx-no-glp-claims.csv', index=False)

In [30]:
med = pd.read_csv('med-no-glp-claims.csv')
rx = pd.read_csv('rx-no-glp-claims.csv')
member = pd.read_csv('no-glp-1-member-claims.csv')

In [31]:
med

,MEDICAL_CLAIM_ID,MEMBER_ID,PAID_DATE,PAID_YEAR,SERVICE_DATE,SERVICE_YEAR,MEDICAL_PAID_AMOUNT,IS_TELEMEDICINE,IS_ER_AVOIDABLE,PRIMARY_DX,...,ICD10_CATEGORY,ICD10_SECTION,MEG_EPISODE_DESCRIPTION,OP_SURG_INC,IS_PCP_VISIT,INCLUDED_SPECIALIST,ER_VISIT_FLAG,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,1915160.00,mma-b66382599c824f5e99482fec1a65c0be,2021-01-01,2021,2020-12-21,2020,0.34,0.00,0.00,Hypertensive chronic kidney disease with stage...,...,Hypertensive diseases,Hypertensive chronic kidney disease,Renal Failure,0,0,0,0,0,0,0
1,2292193.00,mma-b66382599c824f5e99482fec1a65c0be,2021-01-01,2021,2020-12-15,2020,1.39,0.00,0.00,"Chronic kidney disease, stage 4 (severe)",...,Acute kidney failure and chronic kidney disease,Chronic kidney disease (CKD),Anomaly: Defects of Kidney,0,0,0,0,0,0,0
2,1025207.00,mma-f41c6eb76982421f9126a357a7e5f100,2021-01-01,2021,2020-08-11,2020,27.28,0.00,0.00,Prediabetes,...,"Abnormal findings on examination of blood, wit...",Elevated blood glucose level,Diabetes Mellitus Type 2 & Unspec Type Mainten...,0,0,0,0,0,0,0
3,1915160.00,mma-b66382599c824f5e99482fec1a65c0be,2021-01-01,2021,2020-12-21,2020,0.34,0.00,0.00,Hypertensive chronic kidney disease with stage...,...,Hypertensive diseases,Hypertensive chronic kidney disease,Renal Failure,0,0,0,0,0,0,0
4,2673633.00,mma-b66382599c824f5e99482fec1a65c0be,2021-01-01,2021,2020-12-21,2020,4.00,0.00,0.00,Nonrheumatic tricuspid (valve) insufficiency,...,Other forms of heart disease,Nonrheumatic tricuspid valve disorders,Renal Failure,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3880118,1280854.00,mma-2b9ce5d044844a6ab9b7b21911f7609d,2024-12-31,2024,2024-12-18,2024,190.85,0.00,0.00,"Unspecified lump in the left breast, upper inn...",...,Disorders of breast,Unspecified lump in breast,Other Breast Disorders,0,0,0,0,0,0,0
3880119,3404594.00,mma-63a4609b2c394a00b2932195be7adddc,2024-12-31,2024,2024-11-26,2024,749.86,0.00,0.00,"Disorder of adrenal gland, unspecified",...,Disorders of other endocrine glands,Other disorders of adrenal gland,Adrenal Insufficiency,0,0,0,0,0,0,0
3880120,819006.00,mma-af5ce9ec6fdd46a0ad1fdffb2e2c543a,2024-12-31,2024,2024-12-18,2024,5.25,0.00,0.00,Encounter for screening for diabetes mellitus,...,Persons encountering health services for exami...,Encounter for screening for other diseases and...,Encounter for Preventive Health Services,0,0,0,0,0,0,0
3880121,4732981.00,mma-62edd7133abd47c98a82f0b2beae0c08,2024-12-31,2024,2024-11-25,2024,3.55,0.00,0.00,Other artificial openings of urinary tract status,...,Persons with potential health hazards related ...,Artificial opening status,Sleep Disorders,0,0,0,0,0,0,0


In [32]:
rx

,RX_CLAIM_ID,MEMBER_ID,PAID_DATE,SERVICE_DATE,DRUG_NAME_GENERIC,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT,RX_SCRIPT_COUNT,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,PAID_YEAR,SERVICE_YEAR
0,10447358.00,mma-e7157e1e168b40bdbf4df20a21b42afa,2021-01-01,2021-01-02,Lisinopril-hydroCHLOROthiazide,Lisinopril-hydroCHLOROthiazide,0.00,30.00,Cardiac,Antihypertensive Combinations,2021,2021
1,2539206.00,mma-1b6adb4242884af58323e46a8038977f,2021-01-01,2020-12-29,Irbesartan-hydroCHLOROthiazide,Irbesartan-hydroCHLOROthiazide,72.20,90.00,Cardiac,Antihypertensive Combinations,2021,2020
2,2539206.00,mma-1b6adb4242884af58323e46a8038977f,2021-01-01,2020-12-29,Irbesartan-hydroCHLOROthiazide,Irbesartan-hydroCHLOROthiazide,72.20,90.00,Cardiac,Antihypertensive Combinations,2021,2020
3,2539206.00,mma-1b6adb4242884af58323e46a8038977f,2021-01-01,2020-12-29,Irbesartan-hydroCHLOROthiazide,Irbesartan-hydroCHLOROthiazide,72.20,90.00,Cardiac,Antihypertensive Combinations,2021,2020
4,2539206.00,mma-1b6adb4242884af58323e46a8038977f,2021-01-01,2020-12-29,Irbesartan-hydroCHLOROthiazide,Irbesartan-hydroCHLOROthiazide,72.20,90.00,Cardiac,Antihypertensive Combinations,2021,2020
...,...,...,...,...,...,...,...,...,...,...,...,...
13480702,11670907.00,mma-a2e9bf0f7d064007adab31914fb951f3,2024-12-31,2024-12-20,oxyCODONE HCl,oxyCODONE HCl,0.00,5.00,CNS Agents,Opioid Analgesics,2024,2024
13480703,11670907.00,mma-a2e9bf0f7d064007adab31914fb951f3,2024-12-31,2024-12-20,oxyCODONE HCl,oxyCODONE HCl,0.00,5.00,CNS Agents,Opioid Analgesics,2024,2024
13480704,11670907.00,mma-a2e9bf0f7d064007adab31914fb951f3,2024-12-31,2024-12-20,oxyCODONE HCl,oxyCODONE HCl,0.00,5.00,CNS Agents,Opioid Analgesics,2024,2024
13480705,11670907.00,mma-a2e9bf0f7d064007adab31914fb951f3,2024-12-31,2024-12-20,oxyCODONE HCl,oxyCODONE HCl,0.00,5.00,CNS Agents,Opioid Analgesics,2024,2024


In [33]:
rx_wla = rx[(rx['ART_DRUGGRPS_L2'] == 'Weight Loss Agents') | (rx['ART_DRUGGRPS_L1'] == 'GLP-1 Agonists')]
rx_wla.DRUG_NAME_PREFERRED.unique()

array(['Saxenda'], dtype=object)

In [37]:
rx = rx.drop_duplicates()
rx

,RX_CLAIM_ID,MEMBER_ID,PAID_DATE,SERVICE_DATE,DRUG_NAME_GENERIC,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT,RX_SCRIPT_COUNT,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,PAID_YEAR,SERVICE_YEAR
0,10447358.00,mma-e7157e1e168b40bdbf4df20a21b42afa,2021-01-01,2021-01-02,Lisinopril-hydroCHLOROthiazide,Lisinopril-hydroCHLOROthiazide,0.00,30.00,Cardiac,Antihypertensive Combinations,2021,2021
1,2539206.00,mma-1b6adb4242884af58323e46a8038977f,2021-01-01,2020-12-29,Irbesartan-hydroCHLOROthiazide,Irbesartan-hydroCHLOROthiazide,72.20,90.00,Cardiac,Antihypertensive Combinations,2021,2020
7,2327418.00,mma-1b318a20c28d4139b9e99aaddec1d316,2021-01-01,2020-12-31,amLODIPine Besylate,amLODIPine Besylate,0.00,30.00,Cardiac,Calcium Channel Blockers,2021,2020
8,13603270.00,mma-754ba15fa4354f77a25fc05866d40ccc,2021-01-01,2021-01-01,amLODIPine Besylate,amLODIPine Besylate,3.41,30.00,Cardiac,Calcium Channel Blockers,2021,2021
19,711274.00,mma-165d25b9cb9d4a6abc7f0afc10005548,2021-01-01,2020-12-28,Olmesartan Medoxomil,Olmesartan Medoxomil,2.80,17.00,Cardiac,Angiotensin Receptor Blockers,2021,2020
...,...,...,...,...,...,...,...,...,...,...,...,...
13480674,3273020.00,mma-1d61f0051af14b6595b89c089157a8c9,2024-12-31,2024-12-31,Cabergoline,Cabergoline,55.56,8.00,Endocrine,Prolactin Agents,2024,2024
13480675,14309610.00,mma-cfc243071eb0476b8c01ff3c32fab2e2,2024-12-31,2024-10-14,Clopidogrel Bisulfate,Clopidogrel Bisulfate,0.65,30000.00,Hematology,Antiplatelet Agents,2024,2024
13480676,10119718.00,mma-177ec78df96745c58e982f965bc144a5,2024-12-31,2024-12-26,Omeprazole,Omeprazole,2.77,30000.00,Gastrointestinal,Proton Pum Inhibitors,2024,2024
13480679,4138058.00,mma-bea81215f3f84e54ac52f5ea1ac290d0,2024-12-31,2024-12-31,Nirmatrelvir&Ritonavir 300/100,Paxlovid (300/100),1353.10,30.00,NaN,NaN,2024,2024


In [34]:
member = member.drop_duplicates()
member.reset_index(inplace=True)
member.MEMBER_ID.nunique()

20223

In [35]:
med.MEMBER_ID.nunique()

19967

In [38]:
rx.MEMBER_ID.nunique()

19556

In [39]:
# Filter to find members with the specified conditions
members_to_remove = med[(med['ICD10_CHAPTER'] == 'Pregnancy, childbirth and the puerperium') | (med['ICD10_SECTION'] == 'Malignant neoplasm of thyroid gland') |
(med['ICD10_CATEGORY'] == 'Acute pancreatitis') | (med['ICD10_CATEGORY'] == 'Acute kidney failure and chronic kidney disease')]['MEMBER_ID'].unique()
# Combine both lists of members to remove
all_members_to_remove = set(members_to_remove)

# Step 3: Remove those members from the original DataFrames
rx_members_remove = rx[~rx['MEMBER_ID'].isin(all_members_to_remove)]
med_members_remove = med[~med['MEMBER_ID'].isin(all_members_to_remove)]
members_remove = member[~member['MEMBER_ID'].isin(all_members_to_remove)]

In [40]:
med_members_remove.MEMBER_ID.nunique()

18302

In [41]:
rx_members_remove.MEMBER_ID.nunique()

17906

In [42]:
med_agg = med_members_remove.pivot_table(index='MEMBER_ID', columns='PAID_YEAR', values='MEDICAL_PAID_AMOUNT', aggfunc='sum')
med_agg.reset_index(inplace=True)
med_agg.columns.name = None
med_agg.columns = [f"{col}" if col != 'PAID_YEAR' else 'PAID_YEAR' for col in med_agg.columns]
med_agg.rename(columns={'2021': 'Med_2021', '2022': 'Med_2022', '2023': 'Med_2023', '2024': 'Med_2024'}, inplace=True)

In [43]:
med_agg1 = med_agg
med_agg1

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024
0,mma-0001ed26305b48f99696f11dfcea17b7,29731.95,1338.85,4440.36,21202.37
1,mma-00024d2d45224e76915920b058364ff1,180.70,269.09,199.06,680.49
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,5402.26,14290.44,8734.14,5885.90
3,mma-0007e20948f4487d97f4eb2981613ea1,1030.05,844.93,0.00,2181.59
4,mma-0008856832474235be7f58214e0cabc3,489.96,156.80,269.88,197.48
...,...,...,...,...,...
18297,mma-ffe711349c65480c9740ba217d4ad2cc,1376.62,2364.41,2065.03,668.59
18298,mma-ffe7972f15694551af377f4666f57f60,593.13,1291.41,441.64,0.00
18299,mma-ffe9040f10b9429eb7ce09ceaaf1e158,483.38,158.63,493.09,489.41
18300,mma-fff2da3c741f4879991e6f65a74c58e9,440.81,1891.86,2379.53,13742.94


In [44]:
# Define inflation rates (as factors)
inflation_factors = {
    2021: 1.175, 
    2022: 1.117,   
    2023: 1.069, 
    2024: 1
}

# Normalize employer paid amounts for inflation
for year, factor in inflation_factors.items():
    column_name = f'Med_{year}'
    med_agg[column_name] = med_agg[column_name] * factor
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024
0,mma-0001ed26305b48f99696f11dfcea17b7,34935.04,1495.50,4746.74,21202.37
1,mma-00024d2d45224e76915920b058364ff1,212.32,300.57,212.80,680.49
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,6347.66,15962.42,9336.80,5885.90
3,mma-0007e20948f4487d97f4eb2981613ea1,1210.31,943.79,0.00,2181.59
4,mma-0008856832474235be7f58214e0cabc3,575.70,175.15,288.50,197.48
...,...,...,...,...,...
18297,mma-ffe711349c65480c9740ba217d4ad2cc,1617.53,2641.05,2207.52,668.59
18298,mma-ffe7972f15694551af377f4666f57f60,696.93,1442.50,472.11,0.00
18299,mma-ffe9040f10b9429eb7ce09ceaaf1e158,567.97,177.19,527.11,489.41
18300,mma-fff2da3c741f4879991e6f65a74c58e9,517.95,2113.21,2543.72,13742.94


In [45]:
med_agg.fillna(0, inplace=True)
med_agg.isnull().sum()

MEMBER_ID    0
Med_2021     0
Med_2022     0
Med_2023     0
Med_2024     0
dtype: int64

In [46]:
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024
0,mma-0001ed26305b48f99696f11dfcea17b7,34935.04,1495.50,4746.74,21202.37
1,mma-00024d2d45224e76915920b058364ff1,212.32,300.57,212.80,680.49
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,6347.66,15962.42,9336.80,5885.90
3,mma-0007e20948f4487d97f4eb2981613ea1,1210.31,943.79,0.00,2181.59
4,mma-0008856832474235be7f58214e0cabc3,575.70,175.15,288.50,197.48
...,...,...,...,...,...
18297,mma-ffe711349c65480c9740ba217d4ad2cc,1617.53,2641.05,2207.52,668.59
18298,mma-ffe7972f15694551af377f4666f57f60,696.93,1442.50,472.11,0.00
18299,mma-ffe9040f10b9429eb7ce09ceaaf1e158,567.97,177.19,527.11,489.41
18300,mma-fff2da3c741f4879991e6f65a74c58e9,517.95,2113.21,2543.72,13742.94


In [47]:
med_agg['Med_Delta'] = med_agg['Med_2024'] - med_agg['Med_2021']
med_agg['Med_%_Change'] =(( med_agg['Med_2024'] - med_agg['Med_2021'])/ med_agg['Med_2021']) * 100
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change
0,mma-0001ed26305b48f99696f11dfcea17b7,34935.04,1495.50,4746.74,21202.37,-13732.67,-39.31
1,mma-00024d2d45224e76915920b058364ff1,212.32,300.57,212.80,680.49,468.17,220.50
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,6347.66,15962.42,9336.80,5885.90,-461.76,-7.27
3,mma-0007e20948f4487d97f4eb2981613ea1,1210.31,943.79,0.00,2181.59,971.28,80.25
4,mma-0008856832474235be7f58214e0cabc3,575.70,175.15,288.50,197.48,-378.22,-65.70
...,...,...,...,...,...,...,...
18297,mma-ffe711349c65480c9740ba217d4ad2cc,1617.53,2641.05,2207.52,668.59,-948.94,-58.67
18298,mma-ffe7972f15694551af377f4666f57f60,696.93,1442.50,472.11,0.00,-696.93,-100.00
18299,mma-ffe9040f10b9429eb7ce09ceaaf1e158,567.97,177.19,527.11,489.41,-78.56,-13.83
18300,mma-fff2da3c741f4879991e6f65a74c58e9,517.95,2113.21,2543.72,13742.94,13224.99,2553.32


In [48]:
med_agg.fillna(0, inplace=True)
med_agg.isnull().sum()

MEMBER_ID       0
Med_2021        0
Med_2022        0
Med_2023        0
Med_2024        0
Med_Delta       0
Med_%_Change    0
dtype: int64

In [49]:
import numpy as np

# Prepare the years and medical costs
years = np.array([2021, 2022, 2023, 2024])
slope_list = []

# Calculate the slope for each member
for index, row in med_agg.iterrows():
    costs = np.array([row['Med_2021'], row['Med_2022'], row['Med_2023'], row['Med_2024']])
    # Perform linear regression to get the slope
    slope, intercept = np.polyfit(years, costs, 1)  # 1 indicates linear fit
    slope_list.append(slope)

# Add the slope to the DataFrame
med_agg['Med_Slope'] = slope_list
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope
0,mma-0001ed26305b48f99696f11dfcea17b7,34935.04,1495.50,4746.74,21202.37,-13732.67,-39.31,-3794.68
1,mma-00024d2d45224e76915920b058364ff1,212.32,300.57,212.80,680.49,468.17,220.50,131.67
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,6347.66,15962.42,9336.80,5885.90,-461.76,-7.27,-801.09
3,mma-0007e20948f4487d97f4eb2981613ea1,1210.31,943.79,0.00,2181.59,971.28,80.25,197.01
4,mma-0008856832474235be7f58214e0cabc3,575.70,175.15,288.50,197.48,-378.22,-65.70,-102.13
...,...,...,...,...,...,...,...,...
18297,mma-ffe711349c65480c9740ba217d4ad2cc,1617.53,2641.05,2207.52,668.59,-948.94,-58.67,-328.03
18298,mma-ffe7972f15694551af377f4666f57f60,696.93,1442.50,472.11,0.00,-696.93,-100.00,-306.12
18299,mma-ffe9040f10b9429eb7ce09ceaaf1e158,567.97,177.19,527.11,489.41,-78.56,-13.83,11.42
18300,mma-fff2da3c741f4879991e6f65a74c58e9,517.95,2113.21,2543.72,13742.94,13224.99,2553.32,4010.55


In [50]:
rx_agg = rx_members_remove.pivot_table(index='MEMBER_ID', columns='PAID_YEAR', values='RX_PAID_AMOUNT', aggfunc='sum')
rx_agg.reset_index(inplace=True)
rx_agg.columns.name = None
rx_agg.columns = [f"{col}" if col != 'PAID_YEAR' else 'PAID_YEAR' for col in rx_agg.columns]
rx_agg.rename(columns={'2021': 'Rx_2021', '2022': 'Rx_2022', '2023': 'Rx_2023', '2024': 'Rx_2024'}, inplace=True)
rx_agg1 = rx_agg
rx_agg1

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024
0,mma-0001ed26305b48f99696f11dfcea17b7,1248.02,417.37,369.43,671.02
1,mma-00024d2d45224e76915920b058364ff1,21.20,0.00,NaN,NaN
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,0.00,115.78,104.90,630.33
3,mma-0007e20948f4487d97f4eb2981613ea1,83.44,28.65,8.76,37.99
4,mma-0008856832474235be7f58214e0cabc3,15.50,0.00,0.00,20.59
...,...,...,...,...,...
17901,mma-ffe711349c65480c9740ba217d4ad2cc,0.00,NaN,28.74,128.09
17902,mma-ffe7972f15694551af377f4666f57f60,0.00,274.29,152.53,0.00
17903,mma-ffe9040f10b9429eb7ce09ceaaf1e158,135.47,188.65,152.40,109.30
17904,mma-fff2da3c741f4879991e6f65a74c58e9,0.00,24.49,132.93,144.60


In [51]:
# Define inflation rates (as factors)
rx_inflation_factors = {
    2021: 1.31, 
    2022: 1.20,   
    2023: 1.103, 
    2024: 1
}

# Normalize employer paid amounts for inflation
for year, factor in rx_inflation_factors.items():
    column_name = f'Rx_{year}'
    rx_agg[column_name] = rx_agg[column_name] * factor
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024
0,mma-0001ed26305b48f99696f11dfcea17b7,1634.91,500.84,407.48,671.02
1,mma-00024d2d45224e76915920b058364ff1,27.77,0.00,NaN,NaN
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,0.00,138.94,115.70,630.33
3,mma-0007e20948f4487d97f4eb2981613ea1,109.31,34.38,9.66,37.99
4,mma-0008856832474235be7f58214e0cabc3,20.30,0.00,0.00,20.59
...,...,...,...,...,...
17901,mma-ffe711349c65480c9740ba217d4ad2cc,0.00,NaN,31.70,128.09
17902,mma-ffe7972f15694551af377f4666f57f60,0.00,329.15,168.24,0.00
17903,mma-ffe9040f10b9429eb7ce09ceaaf1e158,177.47,226.38,168.10,109.30
17904,mma-fff2da3c741f4879991e6f65a74c58e9,0.00,29.39,146.62,144.60


In [52]:
rx_agg.fillna(0, inplace=True)
rx_agg.isnull().sum()

MEMBER_ID    0
Rx_2021      0
Rx_2022      0
Rx_2023      0
Rx_2024      0
dtype: int64

In [53]:
rx_agg['Rx_Delta'] = rx_agg['Rx_2024'] - rx_agg['Rx_2021']
rx_agg['Rx_%_Change'] =((rx_agg['Rx_2024'] - rx_agg['Rx_2021'])/ rx_agg['Rx_2021']) * 100
# Prepare the years and medical costs
years = np.array([2021, 2022, 2023, 2024])
slope_list = []

# Calculate the slope for each member
for index, row in rx_agg.iterrows():
    costs = np.array([row['Rx_2021'], row['Rx_2022'], row['Rx_2023'], row['Rx_2024']])
    # Perform linear regression to get the slope
    slope, intercept = np.polyfit(years, costs, 1)  # 1 indicates linear fit
    slope_list.append(slope)

# Add the slope to the DataFrame
rx_agg['Rx_Slope'] = slope_list
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-0001ed26305b48f99696f11dfcea17b7,1634.91,500.84,407.48,671.02,-963.89,-58.96,-298.50
1,mma-00024d2d45224e76915920b058364ff1,27.77,0.00,0.00,0.00,-27.77,-100.00,-8.33
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,0.00,138.94,115.70,630.33,630.33,inf,186.78
3,mma-0007e20948f4487d97f4eb2981613ea1,109.31,34.38,9.66,37.99,-71.32,-65.24,-23.87
4,mma-0008856832474235be7f58214e0cabc3,20.30,0.00,0.00,20.59,0.29,1.40,0.09
...,...,...,...,...,...,...,...,...
17901,mma-ffe711349c65480c9740ba217d4ad2cc,0.00,0.00,31.70,128.09,128.09,inf,41.60
17902,mma-ffe7972f15694551af377f4666f57f60,0.00,329.15,168.24,0.00,0.00,NaN,-16.09
17903,mma-ffe9040f10b9429eb7ce09ceaaf1e158,177.47,226.38,168.10,109.30,-68.17,-38.41,-26.28
17904,mma-fff2da3c741f4879991e6f65a74c58e9,0.00,29.39,146.62,144.60,144.60,inf,55.10


In [54]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [55]:
rx_agg.replace(np.inf, 0, inplace=True)
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-0001ed26305b48f99696f11dfcea17b7,1634.91,500.84,407.48,671.02,-963.89,-58.96,-298.50
1,mma-00024d2d45224e76915920b058364ff1,27.77,0.00,0.00,0.00,-27.77,-100.00,-8.33
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,0.00,138.94,115.70,630.33,630.33,0.00,186.78
3,mma-0007e20948f4487d97f4eb2981613ea1,109.31,34.38,9.66,37.99,-71.32,-65.24,-23.87
4,mma-0008856832474235be7f58214e0cabc3,20.30,0.00,0.00,20.59,0.29,1.40,0.09
...,...,...,...,...,...,...,...,...
17901,mma-ffe711349c65480c9740ba217d4ad2cc,0.00,0.00,31.70,128.09,128.09,0.00,41.60
17902,mma-ffe7972f15694551af377f4666f57f60,0.00,329.15,168.24,0.00,0.00,NaN,-16.09
17903,mma-ffe9040f10b9429eb7ce09ceaaf1e158,177.47,226.38,168.10,109.30,-68.17,-38.41,-26.28
17904,mma-fff2da3c741f4879991e6f65a74c58e9,0.00,29.39,146.62,144.60,144.60,0.00,55.10


In [56]:
member_mrg = members_remove.merge(med_agg, on='MEMBER_ID')
member_mrg = member_mrg.merge(rx_agg, on='MEMBER_ID')                                     
member_mrg       

,index,MEMBER_ID,MEMBER_STATUS,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,...,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,1,mma-036ffc20ff794595a44950a4729f66d3,0.00,LA,Subscriber,Female,62,Baby Boomers,1548.18,6302.57,...,346.67,22.39,-466.28,172.55,5632.10,7037.23,774.92,602.37,349.09,321.22
1,3,mma-4c45de73f32445699b80d8d493793dff,0.00,MA,Subscriber,Male,57,Generation X,3612.20,24394.26,...,15832.78,438.31,2989.48,131.86,140.29,384.77,123.66,-8.20,-6.22,21.99
2,9,mma-f8d8a13411e045629b92e106d35b3cd8,0.00,NJ,Spouse,Male,42,Millenials,1302.25,1166.67,...,-912.72,-70.09,-313.69,74.59,0.00,0.00,6.93,-67.66,-90.71,-20.30
3,17,mma-ca2e959e4c094ddab3a345d0d94d2f3e,2.00,GA,Subscriber,Female,65,Baby Boomers,3928.97,241.56,...,-3067.16,-78.07,-891.70,964.85,0.00,0.00,0.00,-964.85,-100.00,-289.46
4,19,mma-ed99a4f01306482caec74ec3d315501c,0.00,IL,Spouse,Female,38,Millenials,24820.37,288.48,...,-23306.72,-93.90,-5735.42,0.00,116.08,19.47,126.69,126.69,0.00,28.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17879,154531,mma-38955c494c7c4c888d0f65459d8959e3,0.00,CO,Subscriber,Female,47,Generation X,1037.07,660.84,...,-54.62,-5.27,-17.30,178.16,244.50,145.34,242.13,63.97,35.91,9.28
17880,154540,mma-46d15af9af544cd6b09133c45ec04642,0.00,WI,Subscriber,Male,47,Generation X,502.23,2096.12,...,73363.57,14607.56,21969.07,52.40,215.66,132.98,3.15,-49.25,-93.99,-23.04
17881,154547,mma-c0b85487143b49dcbd14a6c3794ba398,0.00,DE,Dependent,Female,23,Generation Z,6949.24,2078.82,...,-6949.24,-100.00,-2244.08,0.00,0.00,110.04,169.28,169.28,0.00,61.79
17882,154552,mma-565511f8a56541379d9fd2f16e419df3,0.00,IL,Subscriber,Female,64,Baby Boomers,1326.52,8985.78,...,483.04,36.41,-677.08,244.52,263.88,217.90,70.04,-174.48,-71.36,-56.94


In [57]:
member1 = member_mrg.drop(columns=['MEMBER_STATUS'])
member1

,index,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,...,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,1,mma-036ffc20ff794595a44950a4729f66d3,LA,Subscriber,Female,62,Baby Boomers,1548.18,6302.57,599.78,...,346.67,22.39,-466.28,172.55,5632.10,7037.23,774.92,602.37,349.09,321.22
1,3,mma-4c45de73f32445699b80d8d493793dff,MA,Subscriber,Male,57,Generation X,3612.20,24394.26,6790.70,...,15832.78,438.31,2989.48,131.86,140.29,384.77,123.66,-8.20,-6.22,21.99
2,9,mma-f8d8a13411e045629b92e106d35b3cd8,NJ,Spouse,Male,42,Millenials,1302.25,1166.67,767.91,...,-912.72,-70.09,-313.69,74.59,0.00,0.00,6.93,-67.66,-90.71,-20.30
3,17,mma-ca2e959e4c094ddab3a345d0d94d2f3e,GA,Subscriber,Female,65,Baby Boomers,3928.97,241.56,525.98,...,-3067.16,-78.07,-891.70,964.85,0.00,0.00,0.00,-964.85,-100.00,-289.46
4,19,mma-ed99a4f01306482caec74ec3d315501c,IL,Spouse,Female,38,Millenials,24820.37,288.48,12854.43,...,-23306.72,-93.90,-5735.42,0.00,116.08,19.47,126.69,126.69,0.00,28.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17879,154531,mma-38955c494c7c4c888d0f65459d8959e3,CO,Subscriber,Female,47,Generation X,1037.07,660.84,651.72,...,-54.62,-5.27,-17.30,178.16,244.50,145.34,242.13,63.97,35.91,9.28
17880,154540,mma-46d15af9af544cd6b09133c45ec04642,WI,Subscriber,Male,47,Generation X,502.23,2096.12,1696.09,...,73363.57,14607.56,21969.07,52.40,215.66,132.98,3.15,-49.25,-93.99,-23.04
17881,154547,mma-c0b85487143b49dcbd14a6c3794ba398,DE,Dependent,Female,23,Generation Z,6949.24,2078.82,485.71,...,-6949.24,-100.00,-2244.08,0.00,0.00,110.04,169.28,169.28,0.00,61.79
17882,154552,mma-565511f8a56541379d9fd2f16e419df3,IL,Subscriber,Female,64,Baby Boomers,1326.52,8985.78,765.83,...,483.04,36.41,-677.08,244.52,263.88,217.90,70.04,-174.48,-71.36,-56.94


In [58]:
med_util = med.groupby('MEMBER_ID').agg({
    'DX_IS_CHRONIC': 'max',
    'ER_VISIT_FLAG': 'sum',
    'IS_PCP_VISIT': 'sum',
    'IS_URGENT_CARE_VISIT': 'sum',
    'IS_PREVENTIVE_VISIT': 'sum',
    'IP_ADMIT_INC': 'sum'
}).reset_index()
med_util

,MEMBER_ID,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,mma-0001ed26305b48f99696f11dfcea17b7,1.00,0,6,1,2,19
1,mma-00024d2d45224e76915920b058364ff1,1.00,0,0,0,7,0
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,1.00,0,22,0,4,0
3,mma-0007e20948f4487d97f4eb2981613ea1,1.00,0,18,17,0,0
4,mma-0008856832474235be7f58214e0cabc3,1.00,0,4,4,4,0
...,...,...,...,...,...,...,...
19962,mma-ffe7972f15694551af377f4666f57f60,1.00,0,1,4,2,0
19963,mma-ffe9040f10b9429eb7ce09ceaaf1e158,1.00,0,8,0,2,0
19964,mma-fff2da3c741f4879991e6f65a74c58e9,1.00,0,5,2,4,18
19965,mma-fffb3143f0b84fa690e26308813fceb0,1.00,0,12,1,3,23


In [59]:
merge_member = member1.merge(med_util, on='MEMBER_ID')
merge_member

,index,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,...,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,1,mma-036ffc20ff794595a44950a4729f66d3,LA,Subscriber,Female,62,Baby Boomers,1548.18,6302.57,599.78,...,774.92,602.37,349.09,321.22,1.00,0,10,1,6,0
1,3,mma-4c45de73f32445699b80d8d493793dff,MA,Subscriber,Male,57,Generation X,3612.20,24394.26,6790.70,...,123.66,-8.20,-6.22,21.99,1.00,13,4,2,2,0
2,9,mma-f8d8a13411e045629b92e106d35b3cd8,NJ,Spouse,Male,42,Millenials,1302.25,1166.67,767.91,...,6.93,-67.66,-90.71,-20.30,1.00,0,6,1,4,0
3,17,mma-ca2e959e4c094ddab3a345d0d94d2f3e,GA,Subscriber,Female,65,Baby Boomers,3928.97,241.56,525.98,...,0.00,-964.85,-100.00,-289.46,1.00,0,4,0,1,0
4,19,mma-ed99a4f01306482caec74ec3d315501c,IL,Spouse,Female,38,Millenials,24820.37,288.48,12854.43,...,126.69,126.69,0.00,28.35,1.00,0,7,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17879,154531,mma-38955c494c7c4c888d0f65459d8959e3,CO,Subscriber,Female,47,Generation X,1037.07,660.84,651.72,...,242.13,63.97,35.91,9.28,1.00,0,4,0,5,0
17880,154540,mma-46d15af9af544cd6b09133c45ec04642,WI,Subscriber,Male,47,Generation X,502.23,2096.12,1696.09,...,3.15,-49.25,-93.99,-23.04,1.00,10,0,0,4,0
17881,154547,mma-c0b85487143b49dcbd14a6c3794ba398,DE,Dependent,Female,23,Generation Z,6949.24,2078.82,485.71,...,169.28,169.28,0.00,61.79,1.00,6,12,0,3,0
17882,154552,mma-565511f8a56541379d9fd2f16e419df3,IL,Subscriber,Female,64,Baby Boomers,1326.52,8985.78,765.83,...,70.04,-174.48,-71.36,-56.94,1.00,16,13,0,0,0


In [60]:
merge_member.drop(columns='index', inplace=True)

In [61]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)
merge_member

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,mma-036ffc20ff794595a44950a4729f66d3,LA,Subscriber,Female,62,Baby Boomers,1548.18,6302.57,599.78,1894.85,346.67,22.39,-466.28,172.55,5632.10,7037.23,774.92,602.37,349.09,321.22,1.00,0,10,1,6,0
1,mma-4c45de73f32445699b80d8d493793dff,MA,Subscriber,Male,57,Generation X,3612.20,24394.26,6790.70,19444.98,15832.78,438.31,2989.48,131.86,140.29,384.77,123.66,-8.20,-6.22,21.99,1.00,13,4,2,2,0
2,mma-f8d8a13411e045629b92e106d35b3cd8,NJ,Spouse,Male,42,Millenials,1302.25,1166.67,767.91,389.53,-912.72,-70.09,-313.69,74.59,0.00,0.00,6.93,-67.66,-90.71,-20.30,1.00,0,6,1,4,0
3,mma-ca2e959e4c094ddab3a345d0d94d2f3e,GA,Subscriber,Female,65,Baby Boomers,3928.97,241.56,525.98,861.81,-3067.16,-78.07,-891.70,964.85,0.00,0.00,0.00,-964.85,-100.00,-289.46,1.00,0,4,0,1,0
4,mma-ed99a4f01306482caec74ec3d315501c,IL,Spouse,Female,38,Millenials,24820.37,288.48,12854.43,1513.65,-23306.72,-93.90,-5735.42,0.00,116.08,19.47,126.69,126.69,0.00,28.35,1.00,0,7,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17879,mma-38955c494c7c4c888d0f65459d8959e3,CO,Subscriber,Female,47,Generation X,1037.07,660.84,651.72,982.45,-54.62,-5.27,-17.30,178.16,244.50,145.34,242.13,63.97,35.91,9.28,1.00,0,4,0,5,0
17880,mma-46d15af9af544cd6b09133c45ec04642,WI,Subscriber,Male,47,Generation X,502.23,2096.12,1696.09,73865.80,73363.57,14607.56,21969.07,52.40,215.66,132.98,3.15,-49.25,-93.99,-23.04,1.00,10,0,0,4,0
17881,mma-c0b85487143b49dcbd14a6c3794ba398,DE,Dependent,Female,23,Generation Z,6949.24,2078.82,485.71,0.00,-6949.24,-100.00,-2244.08,0.00,0.00,110.04,169.28,169.28,0.00,61.79,1.00,6,12,0,3,0
17882,mma-565511f8a56541379d9fd2f16e419df3,IL,Subscriber,Female,64,Baby Boomers,1326.52,8985.78,765.83,1809.56,483.04,36.41,-677.08,244.52,263.88,217.90,70.04,-174.48,-71.36,-56.94,1.00,16,13,0,0,0


In [62]:
episodes = med.ICD10_CHAPTER.value_counts()
episodes.head(10)

ICD10_CHAPTER
Factors influencing health status and contact with health services                         788093
Diseases of the musculoskeletal system and connective tissue                               600956
Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified    442493
Endocrine, nutritional and metabolic diseases                                              339513
Mental, Behavioral and Neurodevelopmental disorders                                        210955
Diseases of the circulatory system                                                         202392
Diseases of the genitourinary system                                                       202022
Diseases of the nervous system                                                             167391
Diseases of the digestive system                                                           163603
Diseases of the respiratory system                                                         140852
Name: 

In [63]:
episodes_cost = med.groupby('ICD10_CHAPTER')['MEDICAL_PAID_AMOUNT'].sum().reset_index()
episodes_cost.sort_values(by='MEDICAL_PAID_AMOUNT', ascending=False)

,ICD10_CHAPTER,MEDICAL_PAID_AMOUNT
10,Diseases of the musculoskeletal system and con...,76349045.93
16,Factors influencing health status and contact ...,74186255.83
5,Diseases of the circulatory system,46214797.96
6,Diseases of the digestive system,45361834.75
19,Neoplasms,44753565.34
21,"Symptoms, signs and abnormal clinical and labo...",43113226.25
14,"Endocrine, nutritional and metabolic diseases",37455814.63
9,Diseases of the genitourinary system,37301948.14
11,Diseases of the nervous system,33305126.91
17,"Injury, poisoning and certain other consequenc...",30787867.40


In [64]:
episodes_to_keep = ['Diseases of the musculoskeletal system and connective tissue', 'Neoplasms', 'Diseases of the digestive system', 
                    'Diseases of the circulatory system', 'Diseases of the circulatory system', 'Diseases of the genitourinary system',
                   'Diseases of the nervous system', 'Mental, Behavioral and Neurodevelopmental disorders']
med1 = med
med1['ICD10_CHAPTER'] = med['ICD10_CHAPTER'].apply(lambda x: x if x in episodes_to_keep else 'OTHER')

In [65]:
med_meg = pd.pivot_table(
    med1,
    values='MEDICAL_PAID_AMOUNT',
    index='MEMBER_ID',
    columns='ICD10_CHAPTER',
    aggfunc='sum',
    fill_value=0
)
med_meg.reset_index()
med_meg.columns.name = None
med_meg.drop(columns='OTHER', inplace=True)
med_meg

,Diseases of the circulatory system,Diseases of the digestive system,Diseases of the genitourinary system,Diseases of the musculoskeletal system and connective tissue,Diseases of the nervous system,"Mental, Behavioral and Neurodevelopmental disorders",Neoplasms
MEMBER_ID,,,,,,,
mma-0001ed26305b48f99696f11dfcea17b7,1497.76,0.00,0.00,19534.07,2246.15,0.00,0.00
mma-00024d2d45224e76915920b058364ff1,0.00,0.00,0.00,0.00,0.00,0.00,0.00
mma-0006f22cc2ed4e6da73eb73991bbbf57,964.32,16152.62,5941.18,861.64,0.00,0.00,3557.37
mma-0007e20948f4487d97f4eb2981613ea1,0.00,0.00,0.00,872.44,93.42,0.00,0.00
mma-0008856832474235be7f58214e0cabc3,0.00,0.00,161.01,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...
mma-ffe7972f15694551af377f4666f57f60,0.00,0.00,0.00,0.00,270.09,0.00,0.00
mma-ffe9040f10b9429eb7ce09ceaaf1e158,608.92,0.00,0.00,0.00,0.00,0.00,0.00
mma-fff2da3c741f4879991e6f65a74c58e9,0.00,0.00,11537.00,869.21,0.00,0.00,0.00


In [66]:
member_merge_cond = merge_member.merge(med_meg, on='MEMBER_ID')
member_merge_cond

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC,Diseases of the circulatory system,Diseases of the digestive system,Diseases of the genitourinary system,Diseases of the musculoskeletal system and connective tissue,Diseases of the nervous system,"Mental, Behavioral and Neurodevelopmental disorders",Neoplasms
0,mma-036ffc20ff794595a44950a4729f66d3,LA,Subscriber,Female,62,Baby Boomers,1548.18,6302.57,599.78,1894.85,346.67,22.39,-466.28,172.55,5632.10,7037.23,774.92,602.37,349.09,321.22,1.00,0,10,1,6,0,897.20,0.00,0.00,4493.81,0.00,0.00,0.00
1,mma-4c45de73f32445699b80d8d493793dff,MA,Subscriber,Male,57,Generation X,3612.20,24394.26,6790.70,19444.98,15832.78,438.31,2989.48,131.86,140.29,384.77,123.66,-8.20,-6.22,21.99,1.00,13,4,2,2,0,4326.72,0.00,4601.44,36650.69,0.00,0.00,330.51
2,mma-f8d8a13411e045629b92e106d35b3cd8,NJ,Spouse,Male,42,Millenials,1302.25,1166.67,767.91,389.53,-912.72,-70.09,-313.69,74.59,0.00,0.00,6.93,-67.66,-90.71,-20.30,1.00,0,6,1,4,0,0.00,0.00,0.00,1048.98,0.00,0.00,0.00
3,mma-ca2e959e4c094ddab3a345d0d94d2f3e,GA,Subscriber,Female,65,Baby Boomers,3928.97,241.56,525.98,861.81,-3067.16,-78.07,-891.70,964.85,0.00,0.00,0.00,-964.85,-100.00,-289.46,1.00,0,4,0,1,0,122.01,0.00,447.47,172.32,0.00,0.00,0.00
4,mma-ed99a4f01306482caec74ec3d315501c,IL,Spouse,Female,38,Millenials,24820.37,288.48,12854.43,1513.65,-23306.72,-93.90,-5735.42,0.00,116.08,19.47,126.69,126.69,0.00,28.35,1.00,0,7,0,4,0,514.00,0.00,0.00,1194.08,277.35,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17879,mma-38955c494c7c4c888d0f65459d8959e3,CO,Subscriber,Female,47,Generation X,1037.07,660.84,651.72,982.45,-54.62,-5.27,-17.30,178.16,244.50,145.34,242.13,63.97,35.91,9.28,1.00,0,4,0,5,0,0.00,79.30,5.45,0.00,0.00,0.00,0.00
17880,mma-46d15af9af544cd6b09133c45ec04642,WI,Subscriber,Male,47,Generation X,502.23,2096.12,1696.09,73865.80,73363.57,14607.56,21969.07,52.40,215.66,132.98,3.15,-49.25,-93.99,-23.04,1.00,10,0,0,4,0,0.00,0.00,0.00,64764.14,0.00,0.00,0.00
17881,mma-c0b85487143b49dcbd14a6c3794ba398,DE,Dependent,Female,23,Generation Z,6949.24,2078.82,485.71,0.00,-6949.24,-100.00,-2244.08,0.00,0.00,110.04,169.28,169.28,0.00,61.79,1.00,6,12,0,3,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
17882,mma-565511f8a56541379d9fd2f16e419df3,IL,Subscriber,Female,64,Baby Boomers,1326.52,8985.78,765.83,1809.56,483.04,36.41,-677.08,244.52,263.88,217.90,70.04,-174.48,-71.36,-56.94,1.00,16,13,0,0,0,828.98,123.19,0.00,1383.23,1404.86,0.00,563.15


In [67]:
member_merge_cond.describe()

C:\Users\U1229773\AppData\Local\anaconda3\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,Age,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC,Diseases of the circulatory system,Diseases of the digestive system,Diseases of the genitourinary system,Diseases of the musculoskeletal system and connective tissue,Diseases of the nervous system,"Mental, Behavioral and Neurodevelopmental disorders",Neoplasms
count,17884.00,17884.00,17884.00,17884.00,17884.00,17884.00,17884.00,17884.00,17884.00,17884.00,17884.00,17884.00,17884.00,16019.00,17884.00,17882.00,17884.00,17884.00,17884.00,17884.00,17884.00,17884.00,17884.00,17884.00,17884.00,17884.00,17884.00,17884.00
mean,45.20,6590.50,6742.15,7252.93,7740.31,1149.81,NaN,396.02,1999.17,2357.21,2565.05,2681.95,682.77,-inf,225.62,0.98,8.09,12.18,1.48,3.10,3.23,2121.92,2180.33,1298.63,3791.82,1648.11,1274.03,2204.75
std,15.27,23219.99,26294.52,25361.40,26273.01,31494.71,NaN,10269.10,10067.87,16242.52,16736.52,18112.85,17148.00,NaN,5438.04,0.14,20.57,12.44,3.15,2.89,15.19,19741.17,12005.45,5443.72,19588.61,20264.56,12933.20,21470.61
min,4.00,-55270.45,-141763.50,-60185.32,-28996.22,-925844.38,-inf,-279383.92,-4019.88,-13609.06,-14710.50,-1529.47,-138232.21,-inf,-53925.84,0.00,0.00,0.00,0.00,0.00,0.00,-4539.46,-34848.03,-1976.80,-16530.98,-2549.35,-195.35,-11237.23
25%,36.00,438.31,452.30,454.36,467.45,-1241.19,-67.95,-547.94,4.61,6.58,0.57,0.35,-150.81,-86.85,-55.44,1.00,0.00,4.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,48.00,1262.13,1341.33,1437.75,1495.44,67.28,7.36,25.68,111.78,121.49,124.19,119.10,0.00,-13.45,-0.79,1.00,0.00,9.00,0.00,3.00,0.00,0.00,0.00,0.00,143.43,0.00,0.00,0.00
75%,57.00,4177.64,4385.73,4832.17,5078.55,2013.17,302.02,836.89,552.94,601.42,632.69,667.73,157.92,57.08,58.13,1.00,9.00,16.00,2.00,4.00,0.00,298.56,289.56,266.39,1326.97,201.83,212.53,66.16
max,94.00,929755.23,1480217.63,930552.46,901373.90,899923.17,inf,292190.88,228027.64,1359262.38,1510919.75,1655017.20,1655017.20,25660204289078915072.00,511670.90,1.00,527.00,246.00,117.00,71.00,856.00,1847547.90,603919.46,160709.95,1497293.30,1233568.23,1104950.23,1279992.94


In [68]:
drugs = rx_members_remove.groupby(['ART_DRUGGRPS_L1', 'ART_DRUGGRPS_L2', 'DRUG_NAME_PREFERRED'])['RX_PAID_AMOUNT'].sum().reset_index()
drugs.sort_values(by='RX_PAID_AMOUNT', ascending=False, inplace=True)
drugs

,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT
226,CNS Agents,Anti-Rheumatics/Immune Modulators,Humira (2 Pen),14573394.14
2533,Topical Agents,Psoriasis Agents,Stelara,7970596.87
2487,Topical Agents,Misc. Topical Agents,Dupixent,4761359.63
224,CNS Agents,Anti-Rheumatics/Immune Modulators,Enbrel SureClick,4338532.89
1248,Hematology,Hereditary Angioedema Agents,Ruconest,3634854.60
...,...,...,...,...
690,CNS Agents,Weight Loss Agents,Saxenda,-1234.35
1906,Nutrional,Prenatal Vitamins,PreGenna,-1483.64
830,Chelating Agents,Chelating Agents,Trientine HCl,-6620.11
1870,Musculoskeletal Agents,Muscle Relaxers,Chlorzoxazone,-15693.59


In [69]:
member_merge_cond.to_csv('no-glp1-members-nofilter.csv', index=False)

In [71]:
controlled_members = pd.read_csv('matched_pairs_controlled_members_pdc_nofilter.csv')
controlled_members = controlled_members['MEMBER_ID'].unique()

In [72]:
rx_controlled = rx[rx['MEMBER_ID'].isin(controlled_members)]
rx_controlled

,RX_CLAIM_ID,MEMBER_ID,PAID_DATE,SERVICE_DATE,DRUG_NAME_GENERIC,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT,RX_SCRIPT_COUNT,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,PAID_YEAR,SERVICE_YEAR
1809,5780005.00,mma-10a643cceb7d41d098d15cc1bd9de839,2021-01-01,2020-12-29,Amoxicillin,Amoxicillin,1.46,30.00,Antibiotics,Penicillins,2021,2020
3652,5084.00,mma-3dc4837edfd543bfa723fba865f15cd4,2021-01-02,2021-01-02,Metoprolol Succinate ER,Metoprolol Succinate ER,4.50,90.00,Cardiac,Beta-Blockers,2021,2021
4808,8060244.00,mma-a16c1182f8334534971a19466b7a9f5d,2021-01-03,2021-01-03,lamoTRIgine,lamoTRIgine,0.00,60.00,CNS Agents,Anticonvulsants,2021,2021
5113,14846887.00,mma-7218eaf8d5a64a23a728ea7a41c1a2a3,2021-01-04,2021-01-04,Amitriptyline HCl,Amitriptyline HCl,4.52,60.00,CNS Agents,Antidepressatns - Tricyclics,2021,2021
5368,11667844.00,mma-2296c999ef8d4d2f816ccffd7e47d282,2021-01-04,2021-01-04,Irbesartan-hydroCHLOROthiazide,Irbesartan-hydroCHLOROthiazide,2.74,30.00,Cardiac,Antihypertensive Combinations,2021,2021
...,...,...,...,...,...,...,...,...,...,...,...,...
13478233,1510887.00,mma-9869404381fb4cb9aff7b88ce339c8ad,2024-12-31,2024-12-31,Acyclovir,Acyclovir,61.30,15.00,Topical Agents,Topical Antivirals,2024,2024
13479536,9948447.00,mma-d18899fda1c140f2b4492a936b1ffe46,2024-12-31,2024-12-27,Omeprazole,Omeprazole,0.00,0.00,Gastrointestinal,Proton Pum Inhibitors,2024,2024
13480123,4822292.00,mma-e6a2d4b53c6e425e9c7e88e0286054e1,2024-12-31,2024-12-31,Losartan Potassium-HCTZ,Losartan Potassium-HCTZ,0.00,90.00,Cardiac,Antihypertensive Combinations,2024,2024
13480491,11093707.00,mma-94842869961f4b6083b7decae408ce89,2024-12-31,2024-12-30,Topiramate,Topiramate,0.00,90.00,CNS Agents,Anticonvulsants,2024,2024


In [73]:
med_controlled = med[med['MEMBER_ID'].isin(controlled_members)]
med_controlled

,MEDICAL_CLAIM_ID,MEMBER_ID,PAID_DATE,PAID_YEAR,SERVICE_DATE,SERVICE_YEAR,MEDICAL_PAID_AMOUNT,IS_TELEMEDICINE,IS_ER_AVOIDABLE,PRIMARY_DX,PROCEDURE_DESC,ARTTOS_V2_L1,ARTTOS_V2_L3,DX_IS_CHRONIC,ICD10_CHAPTER,ICD10_CATEGORY,ICD10_SECTION,MEG_EPISODE_DESCRIPTION,OP_SURG_INC,IS_PCP_VISIT,INCLUDED_SPECIALIST,ER_VISIT_FLAG,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
13,3387484.00,mma-0966bfa11a214aedb7cf8078a20063fe,2021-01-01,2021,2020-12-14,2020,32.75,0.00,0.00,Shortness of breath,"Maximum breathing capacity, maximal voluntary ...",Professional,Pulmonology (P37g),0.00,OTHER,Symptoms and signs involving the circulatory a...,Abnormalities of breathing,"Asthma, Chronic Maintenance",0,0,0,0,0,0,0
30,15042239.00,mma-6cb6ef7f2a2048c7a5541746d50ff404,2021-01-01,2021,2020-12-21,2020,87.54,0.00,0.00,"Malignant neoplasm of colon, unspecified",CT of thorax with contrast (CPT 71260),Professional,CT/MRI/PET - CT Scan (P57a),1.00,Neoplasms,Malignant neoplasms of digestive organs,Malignant neoplasm of colon,"Neoplasm, Malignant: Colon and Rectum",0,0,0,0,0,0,0
79,714149.00,mma-bf60fda6efb7468aabbd721da1e4908b,2021-01-01,2021,2020-12-20,2020,100.01,0.00,0.00,Contact with and (suspected) exposure to other...,Cov-19 amp prb hgh thruput (HCPCS U0003),Professional,Office - Independent Lab (P63c),0.00,OTHER,Persons with potential health hazards related ...,Contact with and (suspected) exposure to commu...,Other Viral Infections,0,0,0,0,0,0,0
135,12933387.00,mma-90214768eea44255bda9d17940e6d98b,2021-01-01,2021,2020-12-23,2020,113.85,1.00,0.00,Malignant neoplasm of upper-outer quadrant of ...,Established patient office or other outpatient...,Outpatient,Clinic (O41d),1.00,Neoplasms,Malignant neoplasms of breast,Malignant neoplasm of breast,"Neoplasm, Malignant: Breast, Female",0,0,0,0,0,0,0
193,6035843.00,mma-4642427b19124ac9a86da9edd0dad466,2021-01-01,2021,2020-12-17,2020,1077.33,0.00,0.00,Dermatochalasis of left upper eyelid,Removal of excessive skin and fat of upper eye...,Professional,Outpatient Surgery (P14),0.00,OTHER,"Disorders of eyelid, lacrimal system and orbit",Other disorders of eyelid,Ptosis of Upper Lid,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3876329,1831542.00,mma-ba30c078040b48aaa2508472ea553f0f,2024-12-20,2024,2024-12-11,2024,151.21,1.00,0.00,Encounter for immunization,Established patient office or other outpatient...,Professional,PCP (P32c),0.00,OTHER,Persons with potential health hazards related ...,Encounter for immunization,Encounter for Preventive Health Services,0,1,0,0,0,0,0
3876443,1831542.00,mma-ba30c078040b48aaa2508472ea553f0f,2024-12-20,2024,2024-12-11,2024,54.90,0.00,0.00,Encounter for immunization,"Tetanus, diphtheria toxoids and acellular pert...",Professional,Preventive Immunizations (P41),0.00,OTHER,Persons with potential health hazards related ...,Encounter for immunization,Encounter for Preventive Health Services,0,0,0,0,0,0,0
3876581,1831542.00,mma-ba30c078040b48aaa2508472ea553f0f,2024-12-20,2024,2024-12-11,2024,37.81,0.00,0.00,Encounter for immunization,Subcutaneous administration of a combination v...,Professional,Preventive Immunizations (P41),0.00,OTHER,Persons with potential health hazards related ...,Encounter for immunization,Encounter for Preventive Health Services,0,0,0,0,0,0,0
3878012,13297263.00,mma-32ea1ce8d5bc4dc090887ad507bae96c,2024-12-26,2024,2024-12-23,2024,21.26,1.00,1.00,Local infection of the skin and subcutaneous t...,Established patient office or other outpatient...,Professional,Specialist (P32d),0.00,OTHER,Infections of the skin and subcutaneous tissue,Other local infections of skin and subcutaneou...,Other Inflammations and Infections of Skin and...,0,0,1,0,0,0,0


In [74]:
member_controlled = member[member['MEMBER_ID'].isin(controlled_members)]
member_controlled

,index,MEMBER_ID,MEMBER_STATUS,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS
1,1,mma-036ffc20ff794595a44950a4729f66d3,0.00,LA,Subscriber,Female,62,Baby Boomers
4,17,mma-ca2e959e4c094ddab3a345d0d94d2f3e,2.00,GA,Subscriber,Female,65,Baby Boomers
5,19,mma-ed99a4f01306482caec74ec3d315501c,0.00,IL,Spouse,Female,38,Millenials
6,31,mma-b65786b24666471bb34a35f338d78efc,0.00,PA,Subscriber,Female,55,Generation X
9,41,mma-5c6f80aabb5140e3ad2916c50cfd7cd6,0.00,NC,Subscriber,Female,38,Millenials
...,...,...,...,...,...,...,...,...
1406,11107,mma-a9532facc59f46cea6136cbd96973f0e,0.00,MA,Subscriber,Male,67,Baby Boomers
1450,11448,mma-0b57ccda483b499ab5e8617f0b032cfc,0.00,TX,Subscriber,Female,66,Baby Boomers
1724,13426,mma-4a037934af894450b235f6dcaeb2000a,0.00,WI,Subscriber,Female,30,Millenials
3416,26395,mma-fad37e4ec5734b2f9b20294eb5838eab,0.00,NH,Subscriber,Female,69,Baby Boomers


In [112]:
controlled_members_pdc = pd.read_csv('matched_pairs_controlled_members_pdc.csv')
controlled_members_pdc = controlled_members_pdc['MEMBER_ID'].unique()
member_controlled_pdc = member[member['MEMBER_ID'].isin(controlled_members_pdc)]
med_controlled_pdc = med[med['MEMBER_ID'].isin(controlled_members_pdc)]
rx_controlled_pdc = rx[rx['MEMBER_ID'].isin(controlled_members_pdc)]

In [75]:
import openpyxl

In [76]:
rx_controlled.to_excel('rx-controlled-claims-nofilter.xlsx', index=False, engine='openpyxl')
med_controlled.to_excel('med-controlled-claims-nofilter.xlsx', index=False, engine='openpyxl')
member_controlled.to_excel('member-controlled-claims-nofilter.xlsx', index=False, engine='openpyxl')

In [115]:
rx_controlled_pdc.to_excel('rx_controlled_pdc.xlsx', index=False, engine='openpyxl')
med_controlled_pdc.to_excel('med_controlled_pdc.xlsx', index=False, engine='openpyxl')
member_controlled_pdc.to_excel('member_controlled_pdc.xlsx', index=False, engine='openpyxl')